下載資料套件

In [1]:
import urllib3
import requests
from bs4 import BeautifulSoup

處理資料套件

In [2]:
import pandas as pd
from datetime import datetime
from io import StringIO

載入其他檔案

In [3]:
from headers import *

畫圖套件

In [2]:
import urllib3
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import StringIO
import json
from headers import *
import controllers

import importlib
importlib.reload(controllers)
controller = controllers.FuturesDataController()

In [3]:
url = "https://www.taifex.com.tw/cht/3/futContractsDateAh"
date="2024/09/09"
params = {
             'queryType': '2',
             'goDay': '',
             'doQuery': '1',
             'dateaddcnt': '',
             'queryDate': date
         }
response = requests.post(url,headers=TAIFEX_OI,data=params)
response.raise_for_status()


In [4]:
soup = BeautifulSoup(response.content,'html.parser')
table = soup.table
df = pd.read_html(StringIO(str(table)))
df = df[0]
df

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 交易口數與契約金額  \
   Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1        多方   
                  序 號              商品 名稱                身份別        口數   
0                   1               臺股期貨                自營商      5921   
1                   1               臺股期貨                 投信         0   
2                   1               臺股期貨                 外資     70295   
3                   2               電子期貨                自營商        39   
4                   2               電子期貨                 投信         0   
5                   2               電子期貨                 外資       225   
6                   3             小型臺指期貨                自營商     12861   
7                   3             小型臺指期貨                 投信         0   
8                   3             小型臺指期貨                 外資    136036   
9                   4             微型臺指期貨                自營商     15394   
10                  4             微型臺指期貨                 投信         0   
11                  4             微型臺指期貨                 外資     56514   
12                  5             小型電子期貨                自營商       162   
13                  5             小型電子期貨                 投信         0   
14                  5             小型電子期貨                 外資       418   
15                  6               股票期貨                自營商      1583   
16                  6               股票期貨                 投信         0   
17                  6               股票期貨                 外資      8385   
18                  7              ETF期貨                自營商       177   
19                  7              ETF期貨                 投信         0   
20                  7              ETF期貨                 外資        57   
21                  8            半導體30期貨                自營商        55   
22                  8            半導體30期貨                 投信         0   
23                  8            半導體30期貨                 外資         0   
24                  9          美國標普500期貨                自營商         8   
25                  9          美國標普500期貨                 投信         0   
26                  9          美國標普500期貨                 外資       163   
27                 10        美國那斯達克100期貨                自營商       128   
28                 10        美國那斯達克100期貨                 投信         0   
29                 10        美國那斯達克100期貨                 外資      3077   
30                 11             美國道瓊期貨                自營商        11   
31                 11             美國道瓊期貨                 投信         0   
32                 11             美國道瓊期貨                 外資       668   
33                 12          美國費城半導體期貨                自營商       235   
34                 12          美國費城半導體期貨                 投信         0   
35                 12          美國費城半導體期貨                 外資       190   
36                 13          英國富時100期貨                自營商       176   
37                 13          英國富時100期貨                 投信         0   
38                 13          英國富時100期貨                 外資       195   
39              期貨 小計              期貨 小計                自營商     36750   
40              期貨 小計              期貨 小計                 投信         0   
41              期貨 小計              期貨 小計                 外資    276223   
42               期貨合計               期貨合計               期貨合計    312973   

                                                  
                   空方              多空淨額           
        契約 金額      口數      契約 金額     口數    契約 金額  
0    24921897    5115   21541106    806  3380791  
1           0       0          0      0        0  
2   295548834   68267  287290055   2028  8258779  
3      175740      76     340812    -37  -165072  
4           0       0          0      0        0  
5     1003897     224    1011400      1    -7503  
6    13531363   14476   15240160  -1615 -1708797  
7           0       0          0      0        0  
8   143120816  135905  143095042    131    25774  
9     3243743   16464    3470832

In [5]:
#df.drop(columns = df.columns[0],axis=1,inplace=True)
df.columns=['序號','商品名稱','身份別','多方交易口數','多方交易金額','空方交易口數','空方交易金額','交易淨口數','交易淨金額']
df = df[[col for col in df.columns if col!='序號']]

df_filter = df.loc[df['商品名稱'].isin(['臺股期貨','小型臺指期貨','微型台指期貨'])]

numeric_columns = ['多方交易口數','多方交易金額','空方交易口數','空方交易金額','交易淨口數','交易淨金額']

for col in numeric_columns:
    df_filter[col] = pd.to_numeric(df_filter[col].astype(str).str.replace(',',''),errors='coerce')

df_filter

/var/folders/mm/1vrw2kkx40sc5rljtdlcwtmr0000gn/T/ipykernel_32765/628564386.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[col] = pd.to_numeric(df_filter[col].astype(str).str.replace(',',''),errors='coerce')
/var/folders/mm/1vrw2kkx40sc5rljtdlcwtmr0000gn/T/ipykernel_32765/628564386.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[col] = pd.to_numeric(df_filter[col].astype(str).str.replace(',',''),errors='coerce')
/var/folders/mm/1vrw2kkx40sc5rljtdlcwtmr0000gn/T/ipykernel_32765/6

,商品名稱,身份別,多方交易口數,多方交易金額,空方交易口數,空方交易金額,交易淨口數,交易淨金額
0,臺股期貨,自營商,5921,24921897,5115,21541106,806,3380791
1,臺股期貨,投信,0,0,0,0,0,0
2,臺股期貨,外資,70295,295548834,68267,287290055,2028,8258779
6,小型臺指期貨,自營商,12861,13531363,14476,15240160,-1615,-1708797
7,小型臺指期貨,投信,0,0,0,0,0,0
8,小型臺指期貨,外資,136036,143120816,135905,143095042,131,25774


In [ ]:
df = controller.crawler_taifex_vix("2024/09/09")
df

In [ ]:
date = "2024/09/09"
dates = date.split('/')
params = dates[0]+dates[1].zfill(2)
url = "https://www.taifex.com.tw/file/taifex/Dailydownload/vix/log2data/{params}new.txt"
url

主程式碼

In [1]:
url = "https://www.taifex.com.tw/file/taifex/Dailydownload/vix/log2data/202409new.txt"
response = requests.post(url)
response.encoding = "big5"
df = pd.read_csv(StringIO(response.text),sep=r"\t+",index_col = False,encoding='big5',skiprows=[1],engine='python')
df = df.rename(columns={'交易日期':'date'})
df['date'] = pd.to_datetime(df['date'],format="%Y%m%d")
df = df[['date','臺指選擇權波動率指數']]

NameError: name 'requests' is not defined

In [ ]:
#df = df[['date','臺指選擇權波動率指數']]
df

In [ ]:
url = "https://www.taifex.com.tw/cht/3/futContractsDate"
#querydate = datetime.now().strftime("%Y/%m/%d")
querydate = "2024/08/26"
print(f"爬取日期: {querydate}")
params = {
        'queryType': '2',
        'goDay': '',
        'doQuery': '1',
        'dateaddcnt': '',
        'queryDate': querydate
    }

response = requests.post(url,headers=TAIFEX_OI,data=params)

if response.status_code == 200:
    dfs = pd.read_html(StringIO(response.text))
    if len(dfs) > 0:
        print(len(dfs))
        tdf = dfs[0]
        print("show")
        tdf
    
    else:
        print("沒有找到數據表格")
        
else:
    print(f"請求失敗，狀態碼：{response.status_code}")
        


In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.table
table


In [ ]:
df = pd.read_html(StringIO(str(table)))
df

In [ ]:
print(len(df))
df = df[0]

In [217]:
#df.to_csv('test.csv', index=False, encoding='utf-8-sig')

In [ ]:
df.drop(columns = df.columns[0],axis=1,inplace=True)
df.head()


In [ ]:
df.columns=['商品名稱','身份別','多方交易口數','多方交易金額','空方交易口數','空方交易金額','交易淨口數','交易淨金額','多方未平倉口數','多方未平倉金額','空方未平倉口數','空方未平倉金額','未平倉淨口數','未平倉淨金額']
df

#篩選指定的期貨類型

In [ ]:
df_filter = df.loc[df['商品名稱'].isin(['臺股期貨','小型臺指期貨','微型臺指期貨'])]
df_filter

#載入MongoDB套件

In [221]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi
import json

In [ ]:


# 從配置文件讀取連接信息
with open('config.json') as config_file:
    config = json.load(config_file)

mongo_user = config['mongo_user']
mongo_pass = config['mongo_pass']
mongo_host = config['mongo_host']

# 構建連接 URI
db_uri = f"mongodb+srv://{mongo_user}:{mongo_pass}@{mongo_host}/?retryWrites=true&w=majority&appName=FDB"

# 創建客戶端連接
client = MongoClient(db_uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)



In [ ]:
# 進行數據操作...
client = MongoClient(db_uri, server_api=ServerApi('1'))
db = client['findata']
collection = db['txf_oi']

#檢查是否已存在該日期數據
existing_data = collection.find_one({'date':querydate})
if existing_data:
    print(f"{querydate}的數據已存在,跳過存入")


data_dict = df_filter.to_dict('records')

for record in data_dict:
    record['date'] = datetime.strptime(querydate,"%Y/%m/%d")



result = collection.insert_many(data_dict)
print(f"成功插入 {len(result.inserted_ids)} 條文檔")

client.close()



In [ ]:

# 可選：驗證插入的數據
# 重新連接並查詢數據
client = MongoClient(db_uri, server_api=ServerApi('1'))
db = client['findata']
collection = db['txf_oi']

# 查詢並打印前幾條數據
for doc in collection.find().limit(5):
    print(doc)


client.close()